# 7.1. Разнообразие

Как известно, для формирования случайного леса необходимо разнообразие каждого из решающих деревьев в его составе. Добиться такого разнообразия можно как при помощи варьирования параметров деревьев, так и при помощи варьирования обучающих выборок. Это обычно делается при помощи комбинирования методов случайных подпространств и беггинга. Метод случайных подпространств состоит в случайном выборе подмножества признаков. Бэггинг - это техника отбора подмножества объектов из исходной выборки, которая состоит в последовательном случайном выборе объектов выборки [**с возвращением**](https://ru.wikipedia.org/wiki/Размещение#Размещение_с_повторениями).

Для ясности приведём следующий пример:

Пусть  наша выборка - это черный мешок с пронумерованными шарами. Каждый шар символизирует некоторый объект нашей выборки с соответствующим номером. Процедура бэггинга предлагает нам последовательно хорошо перемешав шары в мешке вытаскивать их один за другим не глядя, записывать их номера на лист бумаги, а затем возвращать их назад в мешок, повторяя эту операцию столько раз, сколько объектов содержится в нашей выборке. Затем мы вынем из мешка все те шары, номер которых хотя бы раз возник в нашем списке. Понятно, что скорее всего среди этих номеров будут повторяющиеся, то есть какие-то шары мы вытянем несколько раз, а какие-то - вообще ни разу. Именно поэтому наша подвыборка и получится случайной. Статистически обоснована оценка того, какой процент шаров из исходной выборки в среднем попадёт в итоговую подвыборку. Эта оценка приблизительно равна $62\%$.

Ваша задача - написать реализацию трёх функций и объединить их в класс `sample`, возвращающий по выборке некоторую случайную подвыборку, пригодную для обучения  одного из деревьев в ансамбле случайного леса.

**Замечание:** обратите внимание, что объекты в итоговой подвыборке не должны дублироваться. Мы предлагаем Вам ознакомиться с функциями [np.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) и [np.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html), они могут оказаться полезны при выполнении этого задания.

Подробнее об этом методе можно прочитать в нашей [лекции](https://colab.research.google.com/drive/1LrqEyfmOKJQdvgxZ56qPcJ_YjJFnP_Ka#scrollTo=kYjii_stqfJo)


In [ ]:
import numpy as np

class sample(object):
  def __init__(self, X, n_subspace):
    self.idx_subspace = self.random_subspace(X, n_subspace)

  def __call__(self, X, y):
    idx_obj = self.bootstrap_sample(X)
    X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
    return X_sampled, y_sampled

  @staticmethod
  def bootstrap_sample(X, random_state=42):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов, выбранных при помощи бэггинга.
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    idx_obj = np.random.choice(X.shape[0], size=X.shape[0], replace=True)
    idx_obj = np.unique(idx_obj)
    return idx_obj

  @staticmethod
  def random_subspace(X, n_subspace, random_state=42):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи метода случайных подпространств признаков
    Количество этих признаков передается при помощи аргумента n_subspace
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    idx_subspace = np.random.choice(X.shape[1], size=n_subspace, replace=False)
    return idx_subspace

  @staticmethod
  def get_subsample(X, y, idx_subspace, idx_obj):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала подвыборку x_sampled, y_sampled
    по значениям индексов признаков(idx_subspace) и объектов(idx_obj) , которые должны в неё попасть
    """
    X_sampled = X[idx_obj][:, idx_subspace]
    y_sampled = y[idx_obj]
    return X_sampled, y_sampled

# Пример формата входных и выходных данных

In [ ]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]])
Y = np.array([1, 2, 3])
s = sample(X, 2)

X_sampled, y_sampled = s.get_subsample(X, Y, s.idx_subspace, s.bootstrap_sample(X))

In [ ]:
# Строки, выбранные из исходного массива X
s.bootstrap_sample(X)

array([0, 2])

In [ ]:
# Столбцы, выбранные из исходного массива X
s.idx_subspace

array([2, 1])

In [ ]:
# Матрица на выходе
X_sampled

array([[3, 2],
       [9, 8]])

In [ ]:
# Вектор ответов на выходе
y_sampled

array([1, 3])

## Примечания

1. Не забывайте, что в качестве ответа на задание нужно отправлять именно реализованный класс, соответствующиий [шаблону выше](https://colab.research.google.com/drive/1LoXr0Rwmcivgla2gAzOKuLgh2ISlokvv#scrollTo=juW_Uoo_ic0z&line=5&uniqifier=1).

2. Подумайте, что должны возвращать методы bootstrap_sample и random_subspace (одна из функций возвращает индексы строк, другая - индексы столбцов).

3. Пример из блокнота не обязан совпадать с входными данными из блокнота. Он дан лишь для проверки алгоритмов в указанном формате.

4. Подумайте, в каких реализуемых методах нужен numpy.unique(), а в каких - нет.

5. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# 7.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле.

- n_estimators - число деревьев в ансамбле
- max_depth - максимальная глубина каждого дерева в ансамбле
- subspaces_dim - размерность случайного подпространства для каждого дерева

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

np.random.seed(42)

N_ESTIMATORS = 10
MAX_DEPTH = 5
SUBSPACE_DIM = 2

class random_forest(object):
  def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
    self.n_estimators = n_estimators
    self.max_depth = max_depth
    self.subspaces_dim = subspaces_dim
    self.random_state = random_state
    self._estimators = []
    self.idx_subspaces = []

  def fit(self, X, y):
    """
      Напишите функцию обучения каждого из деревьев алгоритма на собственной подвыборке
    """
    for i in range(self.n_estimators):
      s = sample(X, self.subspaces_dim)
      idx_subspace = s.random_subspace(X, self.subspaces_dim)
      idx_obj = s.bootstrap_sample(X)
      X_sampled, y_sampled = s.get_subsample(X, y, idx_subspace, idx_obj)

      tree = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
      tree.fit(X_sampled, y_sampled)
      self._estimators.append(tree)
      self.idx_subspaces.append(idx_subspace)

  def predict(self, X):
    """
      Напишите функцию получения среднего предсказания алгоритма
    """
    preds = [tree.predict(X[:, self.idx_subspaces[i]]) for i, tree in enumerate(self._estimators)]
    return np.round(np.mean(preds, axis = 0)).astype(int)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)
rf_model = random_forest(N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM, 42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(y_pred)
print(y_test)
accuracy_score(y_pred, y_test)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]


1.0

## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [ ]:
X = np.array([[ 0.47819456, -1.57891216, -0.1018819 ,  1.11113501,  0.20826281,
        -1.11091227,  0.07844205],
       [ 0.11850997,  1.91073022,  0.95574903,  1.35798262,  0.56177995,
         0.26012021,  0.42404407],
       [-0.52304666,  0.75051167, -1.037804  , -0.10105312,  0.08559063,
         0.5102743 , -1.79068927],
       [-0.09078024,  1.62097709,  0.93284371,  1.0386902 , -0.68354252,
        -1.27138661,  0.15060651],
       [ 0.11676701, -0.71769062, -0.80119565,  0.73448495,  1.80728052,
         0.45770337,  0.20689119]])

In [ ]:
X.shape

(5, 7)

In [ ]:
y.shape

(150,)

In [ ]:
rf = random_forest(25, 15, 2, 42)
rf.fit(X, y)

In [ ]:
rf = random_forest(25, 15, 2, 42)
rf.fit(X, y)

In [ ]:
preds = rf.predict(X)

In [ ]:
preds

array([0, 0, 0, 0, 0])